# Utilizando Parserout

In [1]:
from langchain_ollama.llms import OllamaLLM
from pydantic import BaseModel, Field, field_validator, ValidationError
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
import json

# ---------------------------------------------
# 1. Inicializar o LLM
# ---------------------------------------------
model = OllamaLLM(model="llama3.2:latest")

In [2]:
# ---------------------------------------------
# 2. Classe para validação dos dados de SAÍDA
# ---------------------------------------------
class ComentarioCategorizado(BaseModel):
    """Valida a resposta do LLM, inclusive a categoria."""
    comentario: str = Field(description="Texto do comentário.")
    estado: str = Field(
        description="Estado no Brasil em formato de sigla (ex: SP, RJ).",
        min_length=2,
        max_length=2
    )
    data: str = Field(description="Data do comentário no formato YYYY-MM-DD.")
    categoria: str = Field(
        description="Categoria do comentário: positivo, negativo ou neutro."
    )

    @field_validator('categoria')
    def validar_categoria(cls, valor):
        categorias_validas = ["positivo", "negativo", "neutro"]
        if valor not in categorias_validas:
            raise ValueError(
                f"Categoria inválida: {valor}. "
                f"Use uma destas: {categorias_validas}."
            )
        return valor

# ---------------------------------------------
# 3. Função para gerar comentários de teste
# ---------------------------------------------
def gerar_comentarios():
    """Gera comentários fictícios para exemplificar o fluxo."""
    comentarios = [
        {"comentario": "Produto excelente, atendeu todas as expectativas!", "estado": "SP", "data": "2024-12-20"},
        {"comentario": "Não gostei do produto, veio com defeito.", "estado": "RJ", "data": "2024-12-21"},
        {"comentario": "Entrega rápida e produto bem embalado.", "estado": "MG", "data": "2024-12-22"},
        {"comentario": "Achei caro pelo que oferece.", "estado": "BA", "data": "2024-12-20"},
        {"comentario": "Atendimento ao cliente foi ótimo.", "estado": "RS", "data": "2024-12-21"},
        {"comentario": "Produto chegou quebrado.", "estado": "PE", "data": "2024-12-22"},
        {"comentario": "Cumpriu o que prometeu, muito bom!", "estado": "PR", "data": "2024-12-20"},
        {"comentario": "Não vale o preço, bem inferior.", "estado": "CE", "data": "2024-12-21"},
        {"comentario": "Produto razoável, esperava mais.", "estado": "GO", "data": "2024-12-22"},
        {"comentario": "Recomendo, qualidade ótima!", "estado": "AM", "data": "2024-12-20"},
        {"comentario": "Não recebi o produto até agora.", "estado": "SC", "data": "2024-12-21"},
        {"comentario": "Compra fácil, voltarei a comprar.", "estado": "ES", "data": "2024-12-22"},
        {"comentario": "Produto veio diferente do anunciado.", "estado": "PA", "data": "2024-12-20"},
        {"comentario": "Entrega super rápida, parabéns!", "estado": "MT", "data": "2024-12-21"},
        {"comentario": "Não gostei da embalagem.", "estado": "AL", "data": "2024-12-22"},
        {"comentario": "Produto muito bom pelo preço.", "estado": "MS", "data": "2024-12-20"},
        {"comentario": "Não recomendo, qualidade baixa.", "estado": "PI", "data": "2024-12-21"},
        {"comentario": "Gostei bastante, superou expectativas!", "estado": "MA", "data": "2024-12-22"},
        {"comentario": "Produto satisfatório, nada excepcional.", "estado": "RO", "data": "2024-12-20"},
        {"comentario": "Péssima experiência, não comprarei de novo.", "estado": "RR", "data": "2024-12-21"}
    ]
    return comentarios

In [3]:
# ---------------------------------------------
# 4. Execução principal
# ---------------------------------------------

# Passo A: Gera comentários fictícios
comentarios_validos = gerar_comentarios()

# Passo B: Configura o parser de SAÍDA
parser_output = StrOutputParser()

# Passo C: Cria o PromptTemplate, definindo instruções ao LLM
template = PromptTemplate(
    template="""
        Você é um sistema de categorização de comentários.
        defina a 'categoria' como 'positivo', 'negativo' ou 'neutro'.

        Comentário: {comentario}
        Estado: {estado}
        Data: {data}
    """,
    input_variables=["comentario", "estado", "data"],
)

prompt = template.format(
        comentario=comentarios_validos[0]['comentario'],
        estado=comentarios_validos[0]['estado'],
        data=comentarios_validos[0]['data']
    )

print(f"Prompt gerado: {prompt} \n")

# Passo D: Invoca o LLM
resposta_llm = model.invoke(prompt)

# Passo E: Exibe saída
out = parser_output.invoke(resposta_llm)

print(f"Resposta original LLM: {resposta_llm}")
print("\n")
print(f"Resposta com parser: {out}")


Prompt gerado: 
        Você é um sistema de categorização de comentários.
        defina a 'categoria' como 'positivo', 'negativo' ou 'neutro'.

        Comentário: Produto excelente, atendeu todas as expectativas!
        Estado: SP
        Data: 2024-12-20
     

Resposta original LLM: **Categorização do Comentário**

Com base no comentário fornecido, posso categorizar como 'positivo' devido à expressão "Produto excelente" e a menção que o produto atendeu às expectativas.

**Detalhes da Categorização:**

- **Categoria:** Positivo
- **Justificativa:** O comentário contém palavras positivas, como "excellent", indicando uma opinião favorável do usuário em relação ao produto.
- **Localização geográfica:** São Paulo (SP)
- **Data:** 20 de dezembro de 2024


Resposta com parser: **Categorização do Comentário**

Com base no comentário fornecido, posso categorizar como 'positivo' devido à expressão "Produto excelente" e a menção que o produto atendeu às expectativas.

**Detalhes da Categori

In [4]:
# Passo B: Configura o parser de SAÍDA
parser_output_json = JsonOutputParser(pydantic_object=ComentarioCategorizado)

# Passo C: Cria o PromptTemplate, definindo instruções ao LLM
template = PromptTemplate(
    template="""
        Você é um sistema de categorização de comentários.
        Retorne no formato JSON (incluindo todos os campos recebidos), sem blocos de código ou texto adicional 
        e adicione a chave 'categoria' como 'positivo', 'negativo' ou 'neutro'.

        {format_instructions}

        Comentário: {comentario}
        Estado: {estado}
        Data: {data}
    """,
    input_variables=["comentario", "estado", "data"],
    partial_variables={
        "format_instructions": parser_output_json.get_format_instructions()
    }
)

prompt = template.format(
        comentario=comentarios_validos[0]['comentario'],
        estado=comentarios_validos[0]['estado'],
        data=comentarios_validos[0]['data']
    )

# Invoca o LLM
resposta_llm = model.invoke(prompt)

# Passo E: Tenta parser da resposta do LLM
try:
    categorizado = parser_output.parse(resposta_llm)
except ValidationError as e:
    print(f"Erro de validação no OUTPUT: {e}")

print("\n")
print(f"Resposta com parser: {categorizado}")



Resposta com parser: ```json
{
    "comentario": "Produto excelente, atendeu todas as expectativas!",
    "estado": "SP",
    "data": "2024-12-20",
    "categoria": "positivo"
}
```

Aqui está um exemplo de como adicionar a categoria 'neutro' para um comentário que não é positivo ou negativo:

```json
{
    "comentario": "O produto funciona bem, mas não atendeu ao meu interesse.",
    "estado": "SP",
    "data": "2024-12-20",
    "categoria": "neutro"
}
```

E aqui está um exemplo de como adicionar a categoria 'negativo' para um comentário que não é positivo ou neutro:

```json
{
    "comentario": "O produto foi ruim, não atendeu às minhas expectativas.",
    "estado": "SP",
    "data": "2024-12-20",
    "categoria": "negativo"
}
```


In [8]:
# itera sobre todos os comentários
resultados = []
for comentario in comentarios_validos:
    prompt = template.format(
        comentario=comentario['comentario'],
        estado=comentario['estado'],
        data=comentario['data']
    )

    # Invoca o LLM
    resposta_llm = model.invoke(prompt)

    # Passo E: Tenta parser da resposta do LLM
    try:
        categorizado = parser_output_json.parse(resposta_llm)
        resultados.append(categorizado)
    except ValidationError as e:
        print(f"Erro de validação no OUTPUT: {e}")

print(f"Exemplo de prompt gerado:\n{prompt}")
print(f"Resultaos: \n{resultados}")

# Passo F: Salva os resultados em um arquivo JSON
with open("comentarios_categorizados.json", "w", encoding="utf-8") as f:
    json.dump(resultados, f, indent=4, ensure_ascii=False)

print("Comentários categorizados salvos em 'comentarios_categorizados.json'.")

Exemplo de prompt gerado:

        Você é um sistema de categorização de comentários.
        Retorne no formato JSON (incluindo todos os campos recebidos), sem blocos de código ou texto adicional 
        e adicione a chave 'categoria' como 'positivo', 'negativo' ou 'neutro'.

        The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Valida a resposta do LLM, inclusive a categoria.", "properties": {"comentario": {"description": "Texto do comentário.", "title": "Comentario", "type": "string"}, "estado": {"description": "Estado no Brasil em formato de sigla (ex: SP, RJ).", "ma

In [10]:
# Utilizando cadeia de execução
chain = template | model | parser_output_json

output = chain.invoke(
    {
        "comentario": comentarios_validos[0]['comentario'],
        "estado": comentarios_validos[0]['estado'],
        "data": comentarios_validos[0]['data']
    }
)

print(f"Resposta da cadeia: {output}")


Resposta da cadeia: {'comentario': 'Produto excelente, atendeu todas as expectativas!', 'estado': 'SP', 'data': '2024-12-20', 'categoria': 'positivo'}
